# Continued pretraining (CP) using Unsloth

In [1]:
#install latest version of unsloth
#%pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

#%pip install --upgrade unsloth
%pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
%pip install --upgrade unsloth_zoo
%pip install wandb

In [1]:
from unsloth import FastLanguageModel, is_bfloat16_supported

from unsloth import UnslothTrainingArguments, UnslothTrainer
from datasets import load_dataset

import torch

# random state
SEED = 42

# reproducibility
## torch
torch.manual_seed(SEED)

## python
import random
random.seed(SEED)

## numpy
import numpy as np
np.random.seed(SEED)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## 🧠 Load Model

In [2]:
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model_id = "meta-llama/Llama-3.2-1B"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.381 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

## Init LoRA

In [3]:
from peft import LoftQConfig

"""
    This is the sub-configuration class to store the configuration of a [`LoraModel`].

    Args:
        bits_pattern (`dict`): The mapping from layer names or regexp expression to bits which are different from the
            default bits specified by `bits`. For example, `{model.decoder.layers.0.encoder_attn.k_proj: 2`}.
        bits (`int`): Quantization bits for LoftQ.
        iter (`int`): Alternating iterations for LoftQ.
        fake (`bool`): True: use fp16/fp32; used for first time to save weights. False: use bitsandbytes 4bit linear
            models. weights can't be saved. Recommend to set to True, save the weights and load the saved weights in 4
            bits.
    """

loftq_config = LoftQConfig(loftq_bits=4)

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 256, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "embed_tokens", "lm_head"], #now also embeddings and lm_head
    lora_alpha = 1,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = SEED,
    use_rslora = True,  # We support rank stabilized LoRA
    init_lora_weights = "loftq",
    loftq_config = loftq_config, # And LoftQ
)

Unsloth: Offloading input_embeddings to disk to save VRAM


/home/mlynatom/master-thesis-repository-tomas-mlynar/venv/master_venv/lib/python3.12/site-packages/unsloth/models/_utils.py:760: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2025.2.15 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


## Prepare Data

In [ ]:
# from datasets import load_dataset

# # this dataset has already fixed encoding using ftfy (as is used by me in the preprocessing steps of other datasets)
# dataset_cs = load_dataset("HuggingFaceFW/fineweb-2", "ces_Latn", split="train")
# #we need only texts
# dataset_cs = dataset_cs.remove_columns(["id", "dump", "url", "date", "file_path", "language", "language_score", "language_script", "minhash_cluster_size", "top_langs"])
# #shuffle to be sure we select "random sample"
# dataset_cs = dataset_cs.shuffle(seed=42)
# dataset_cs

In [ ]:
# dataset_en = load_dataset("HuggingFaceFW/fineweb-edu", "sample-10BT", split="train")
# dataset_en


In [ ]:
# dataset_en = dataset_en.remove_columns(["id", "dump", "url", "file_path", "language", "language_score", "token_count", "score", "int_score"])
# dataset_en.shuffle(seed=42)
# dataset_en

In [ ]:
# from datasets import concatenate_datasets
# #select 40000 samples (for the initial exp.)

# dataset_cs = dataset_cs.select(range(30000))
# dataset_en = dataset_en.select(range(10000))

# dataset = concatenate_datasets([dataset_cs, dataset_en])
# dataset

In [ ]:
# dataset.save_to_disk("data/pretraining/init_31mix_cs-en")

In [7]:
from datasets import load_from_disk
dataset = load_from_disk("data/pretraining/init_31mix_cs-en")
dataset

Dataset({
    features: ['text'],
    num_rows: 40000
})

In [8]:
def preprocess_function(examples):
    return {"text": [example + tokenizer.eos_token for example in examples["text"]]}

dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

In [10]:
for row in dataset[:5]["text"]:
    print("=========================")
    print(row)

V dnešní době je wifi připojení doma či v práci nezbytností. Ať už pracujete z domova, streamujete filmy nebo hrajete online hry, stabilní a rychlé připojení je klíčem k úspěchu. Frekvence wifi signálu je jedním z faktorů, který může ovlivnit rychlost a stabilitu připojení. V tomto článku se zaměříme na pět kroků, které vám pomohou optimálně nastavit frekvenci vašeho wifi signálu.
Krok 1: Vyberte správný kanál
Naše wifi sítě fungují na dvou frekvencích – 2,4 GHz a 5 GHz. Každá z těchto frekvencí má určité množství kanálů. Na frekvenci 2,4 GHz máme k dispozici 14 kanálů. Kanály jsou však překrývající se, což znamená, že pokud jsou dva sousedící kanály použity, mohou si vzájemně rušit. Proto se v praxi používají převážně kanály 1, 6 a 11.
Na frekvenci 5 GHz je k dispozici více kanálů a ty jsou navíc nezávislé, tedy se nepřekrývají. To znamená, že na této frekvenci můžeme dosáhnout vyšší přenosové rychlosti.
Krok 2: Změřte rušení
Pokud chcete zjistit, který kanál je pro vás nejlepší, může

## Train the model

In [ ]:
class UnslothKLLossTrainer(UnslothTrainer):
    def __init__(self, ref_model, kl_weight=0.1, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.ref_model = ref_model
        self.ref_model.eval()
        self.kl_weight = kl_weight

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        #account for the actual number of items in the batch
        if num_items_in_batch is not None:
            effective_batch_size = num_items_in_batch
        else:
            effective_batch_size = labels.size(0)
        
        #cross entropy loss
        loss_fct = torch.nn.CrossEntropyLoss(reduction="sum")
        loss_ce = loss_fct(logits.view(-1, logits.shape[-1]), labels.view(-1))
        loss_ce = loss_ce / effective_batch_size # Scale by actual number of items

        # KL loss
        with torch.no_grad():
            ref_outputs = self.ref_model(**inputs)
            ref_logits = ref_outputs.logits
        
        loss_kl_fct = torch.nn.KLDivLoss(reduction="sum")
        loss_kl = loss_kl_fct(
            torch.nn.functional.log_softmax(logits, dim=-1),
            torch.nn.functional.softmax(ref_logits, dim=-1),
        )
        loss_kl = loss_kl / effective_batch_size # Scale by actual number of items

        total_loss = loss_ce + self.kl_weight*loss_kl

        return (total_loss, outputs) if return_outputs else total_loss

from transformers.utils.import_utils import is_torch_xla_available
#from transformers.trainer_utils import SaveStrategy
from typing import Dict
import numpy as np

class PerplexityTrainer(UnslothTrainer):
    # def training_step(self, model, inputs, num_items_in_batch):
    #     loss = super().training_step(model, inputs, num_items_in_batch)

    #     # log perplexity
    #     perplexity = torch.exp(loss)
    #     self.log({"train_perplexity": perplexity})

    #     return loss
    def _maybe_log_save_evaluate(self, tr_loss, grad_norm, model, trial, epoch, ignore_keys_for_eval):
        if self.control.should_log and self.state.global_step > self._globalstep_last_logged:
            if is_torch_xla_available():
                xm.mark_step()

            logs: Dict[str, float] = {}

            # all_gather + mean() to get average loss over all processes
            tr_loss_scalar = self._nested_gather(tr_loss).mean().item()

            # reset tr_loss to zero
            tr_loss -= tr_loss

            loss = round(tr_loss_scalar / (self.state.global_step - self._globalstep_last_logged), 4)
            logs["loss"] = loss

            #PERPLEXITY
            logs["perplexity"] = round(float(np.exp(tr_loss_scalar)), 4)
            #########################


            if grad_norm is not None:
                logs["grad_norm"] = grad_norm.detach().item() if isinstance(grad_norm, torch.Tensor) else grad_norm
            logs["learning_rate"] = self._get_learning_rate()

            self._total_loss_scalar += tr_loss_scalar
            self._globalstep_last_logged = self.state.global_step
            self.store_flos()

            self.log(logs)

        metrics = None
        if self.control.should_evaluate:
            metrics = self._evaluate(trial, ignore_keys_for_eval)

        if self.control.should_save:
            self._save_checkpoint(model, metrics=metrics)
            self.control = self.callback_handler.on_save(self.args, self.state, self.control)

TrainerClass = PerplexityTrainer

In [9]:
from transformers import EvalPrediction
import numpy as np

#prepare perplexity computation
def compute_metrics(pred: EvalPrediction):
    losses = pred.losses

    mean_loss = np.mean(losses)

    try:
        perplexity = np.exp(mean_loss)
    except OverflowError:
        perplexity = float("inf")

    return {"perplexity": perplexity}


In [11]:
import copy

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset.shuffle(seed=SEED),
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    # ref_model=copy.deepcopy(model),
    #compute_metrics=compute_metrics,
    
    args = UnslothTrainingArguments(
        per_device_train_batch_size = 64,
        gradient_accumulation_steps = 16,
        warmup_ratio = 0.05,
        num_train_epochs = 1, # Set this for 1 full training run.
        #max_steps = 1000,
        learning_rate = 2e-4,
        embedding_learning_rate = 4e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = SEED,
        output_dir = "outputs",
        report_to = "wandb", # Use this for WandB etc
        run_name="llama3.2-1b-cp-exp-loftq",
        # eval_strategy = "steps",
        # eval_steps = 50,
        
    ),
)

/home/mlynatom/master-thesis-repository-tomas-mlynar/unsloth_compiled_cache/UnslothSFTTrainer.py:578: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/mlynatom/master-thesis-repository-tomas-mlynar/unsloth_compiled_cache/UnslothSFTTrainer.py:592: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/mlynatom/master-thesis-repository-tomas-mlynar/unsloth_compiled_cache/UnslothSFTTrainer.py:606: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map (num_proc=2):   0%|          | 0/40000 [00:00<?, ? examples/s]

In [12]:
#Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.381 GB.
3.514 GB of memory reserved.


In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 40,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 64 | Gradient Accumulation steps = 16
\        /    Total batch size = 1,024 | Total steps = 39
 "-____-"     Number of trainable parameters = 705,691,648


Unsloth: Setting lr = 4.00e-05 instead of 2.00e-04 for embed_tokens.
Unsloth: Setting lr = 4.00e-05 instead of 2.00e-04 for lm_head.


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mlynatom. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,2.909600
2,2.845400
3,2.885000
4,2.875000


/home/mlynatom/master-thesis-repository-tomas-mlynar/venv/master_venv/lib/python3.12/site-packages/unsloth/models/_utils.py:592: SyntaxWarning: invalid escape sequence '\.'
  source = re.sub("([^\.])nn\.", r"\1torch.nn.", source)
/home/mlynatom/master-thesis-repository-tomas-mlynar/venv/master_venv/lib/python3.12/site-packages/unsloth/models/_utils.py:855: SyntaxWarning: invalid escape sequence '\)'
  "self.rotary_emb = .+?\)", function,
/home/mlynatom/master-thesis-repository-tomas-mlynar/venv/master_venv/lib/python3.12/site-packages/unsloth/models/_utils.py:955: SyntaxWarning: invalid escape sequence '\)'
  "self.rotary_emb = .+?\)", function,


KeyboardInterrupt: 

In [15]:
#Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

4329.0353 seconds used for training.
72.15 minutes used for training.
Peak reserved memory = 19.932 GB.
Peak reserved memory for training = 17.297 GB.
Peak reserved memory % of max memory = 50.613 %.
Peak reserved memory for training % of max memory = 43.922 %.
